# BSE Trend analysis using Pandas

### Step 1 : Scope the project & gather data

#### Scope

Trend analysis on 1Yr worth of BSE data

#### Installs

#### Imports

In [1]:
# Default Installs
import pandas as pd
import os
import numpy as np
import datetime as dt

In [2]:
# Manual Installs
from tabulate import tabulate

##### Environments

In [3]:
print('getcwd : ', os.getcwd())

getcwd :  D:\BigData\12. Python\4. Jupyter Notebooks


##### Pandas options

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# for all rows set to None
# pd.set_option('display.max_rows', None) 

pd.set_option('display.max_rows', 100)

#### Describe and gather data

##### Things to remember
* Data from mySQL should be a whole weeks data not any partial in between data

In [5]:
%%time
bse_csv_columns = ["ts", "sc_code", "sc_name", "sc_group", "sc_type"
                 , "open", "high", "low", "close", "last", "prevclose"
                 , "no_trades", "no_of_shrs", "net_turnover", "tdcloindi", "isin"]

df_bse_daily = pd.read_csv(os.path.join(os.getcwd(), '..', '5. BTD', 'data', 'bse_daily_365d.csv'), sep='|'
                           , names=bse_csv_columns
                          ,skip_blank_lines=True
                          ,parse_dates=['ts'])

Wall time: 3.37 s


In [8]:
print(df_bse_daily.shape)

(657058, 16)


#### Python functions

### Step 2 : Explore and assess the data

In [33]:
df_bse_daily['net_turnover'].sum()

6506680143163

In [109]:
df_bse_daily.groupby(['sc_group'])['sc_group'].count()

sc_group
A     114330
B     239492
E       2667
F       1150
IF       669
M      15685
MS        66
MT       564
P       1030
R         18
T      30339
TS        33
X     177775
XT     62820
Z      10391
ZP        29
Name: sc_group, dtype: int64

Using ABB as thats the first company in the list

In [14]:
%%time
df_temp = df_bse_daily[df_bse_daily.sc_code == 500002][['ts', 'sc_code', 'sc_name', 'high', 'low', 'close', 'prevclose', 'no_of_shrs']]

Wall time: 18.1 ms


In [15]:
print(df_temp.columns)
df_temp.info()

Index(['ts', 'sc_code', 'sc_name', 'high', 'low', 'close', 'prevclose',
       'no_of_shrs'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 0 to 654560
Data columns (total 8 columns):
ts            245 non-null datetime64[ns]
sc_code       245 non-null int64
sc_name       245 non-null object
high          245 non-null float64
low           245 non-null float64
close         245 non-null float64
prevclose     245 non-null float64
no_of_shrs    245 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 16.3+ KB


In [16]:
df_temp.head()

,ts,sc_code,sc_name,high,low,close,prevclose,no_of_shrs
0,2019-04-01,500002,ABB LTD.,1326.60,1306.35,1313.55,1319.85,7785
2791,2019-04-02,500002,ABB LTD.,1342.25,1314.00,1322.60,1313.55,4482
5519,2019-04-03,500002,ABB LTD.,1339.95,1319.70,1329.00,1322.60,27791
8294,2019-04-04,500002,ABB LTD.,1336.10,1317.30,1326.00,1329.00,3106
11005,2019-04-05,500002,ABB LTD.,1379.00,1320.30,1374.75,1326.00,12887


**Observations**
* column no_of_shrs needs to be renamed. Changing it to volumes
* Adding calculated columns like change(close- prevclose), change percent( ((close/prevclose)*100) - 100)
* Since i am calculating weekly analysis, adding a new column called yearWeek which contain year and week number

In [17]:
# Renaming column
df_temp.rename(columns={"no_of_shrs": "volumes"}, inplace=True)

In [17]:
# Last week of year comes as 0 because that week is also beginning of next year
#df_temp['yearWeek'] = df_temp['ts'].dt.year * 100 + df_temp['ts'].dt.week+ 0

# This resolves above problem
df_temp['yearWeek'] = df_temp['ts'].dt.year * 100 + df_temp['ts'].dt.strftime('%U').astype(int)

In [18]:
# Change & change percent
df_temp['chng'] = (df_temp['close'] - df_temp['prevclose'])
df_temp['chngp'] = np.round( ( (df_temp['close'] / df_temp['prevclose']) * 100 ) - 100, 2)

In [19]:
# New Years Weeknumber is 00
df_temp[df_temp.yearWeek>201951].head(10)

,ts,sc_code,sc_name,high,low,close,prevclose,volumes,yearWeek,chng,chngp
487188,2019-12-30,500002,ABB LTD.,1310.00,1260.00,1287.35,1275.75,1492,201952,11.60,0.91
489968,2019-12-31,500002,ABB LTD.,1305.00,1275.00,1284.10,1287.35,1554,201952,-3.25,-0.25
492740,2020-01-01,500002,ABB LTD.,1297.00,1280.00,1287.00,1284.10,493,202000,2.90,0.23
495374,2020-01-02,500002,ABB LTD.,1304.00,1284.00,1294.65,1287.00,884,202000,7.65,0.59
498083,2020-01-03,500002,ABB LTD.,1324.90,1285.00,1321.45,1294.65,2424,202000,26.80,2.07
500813,2020-01-06,500002,ABB LTD.,1330.00,1276.45,1286.70,1321.45,3409,202001,-34.75,-2.63
503559,2020-01-07,500002,ABB LTD.,1315.10,1290.10,1307.35,1286.70,2306,202001,20.65,1.60
506238,2020-01-08,500002,ABB LTD.,1317.70,1287.20,1309.65,1307.35,2312,202001,2.30,0.18
508895,2020-01-09,500002,ABB LTD.,1331.85,1293.70,1319.40,1309.65,2876,202001,9.75,0.74
511691,2020-01-10,500002,ABB LTD.,1411.00,1335.00,1367.25,1319.40,7756,202001,47.85,3.63


##### Analysis data & Indicators

Below are the possible indicators, i can think of,
1. closeH - high close in the week
1. closeL - low closein the week
1. volHigh - Highest volume in the week
1. volAvg - Volume average
1. daysTraded - Number of days traded in the week
1. HSDL - Highest Single Day Loss
1. HSDG - Highest Single Day Gain
1. HSDLp - Highest Single Day Loss percent
1. HSDGp - Highest Single Day Gain percent
1. first - First close of the week
1. last - Last close of he week
1. wChng - Week change
1. wChngp - Week change percent
1. lastTrdDoW - Last traded day of week
1. TI - Times increased
1. volAvgWOhv - Volume average without high volume
1. HVdAV - High volume / Average volume(without highvolume)
1. CPveoHVD - Close positive on high volume day
1. lastDVotWk - Last day volume
1. lastDVdAV - Last day volume / average volume


**Calculating data for columns closeL, closeH, volAvg(of all days in week) and volHigh**

In [200]:
%%time
# close(High/Low), volume average of all days in week, highest volume of the week
df_temp2 = df_temp.groupby(['sc_code', 'yearWeek'], sort=False)['close'].agg([('L','min'),('H', 'max')]).add_prefix('close').reset_index().copy()
df_temp3 = df_temp.groupby(['sc_code', 'yearWeek'], sort=False)['volumes'].agg([('Avg','mean'),('High', 'max')]).add_prefix('vol').reset_index().copy()
df_temp3['volAvg']= df_temp3['volAvg'].astype(int)
df_temp3 = pd.merge(df_temp2, df_temp3, how='left', on=['sc_code', 'yearWeek'])
#df_grouped = df_grouped.rename(columns={'close':'hClose'})
df_weekly = df_temp3.copy()

Wall time: 70.6 ms


In [201]:
df_weekly.head()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh
0,500002,201913,1313.55,1374.75,11210,27791
1,500002,201914,1379.05,1426.20,16741,34789
2,500002,201915,1408.60,1432.30,9008,21127
3,500002,201916,1433.05,1493.90,23703,50389
4,500002,201917,1473.40,1490.55,13102,27577


**Calculating daysTraded**

In [202]:
%%time
# Count number of trading days in the week
df_temp2 = df_temp.groupby(['sc_code', 'yearWeek'], sort=False).count().reset_index()[['sc_code', 'yearWeek', 'ts']]
df_temp2.columns = ['sc_code', 'yearWeek', 'daysTraded']
df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=['sc_code', 'yearWeek'])

Wall time: 27.9 ms


**Calculating HSDL, HSDG, HSDLp and HSDGp**

In [203]:
%%time
# High Single Day chng/chngp
df_temp2 = df_temp.groupby(['sc_code', 'yearWeek'], sort=False)['chng'].agg([('L','min'),('G', 'max')]) \
            .add_prefix('HSD').reset_index().copy()
df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=['sc_code', 'yearWeek'])

df_temp2 = df_temp.groupby(['sc_code', 'yearWeek'], sort=False)['chngp'].agg([('Lp','min'),('Gp', 'max')]) \
            .add_prefix('HSD').reset_index().copy()
df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=['sc_code', 'yearWeek'])

Wall time: 48 ms


In [204]:
df_weekly.head()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,HSDL,HSDG,HSDLp,HSDGp
0,500002,201913,1313.55,1374.75,11210,27791,5,-6.30,48.75,-0.48,3.68
1,500002,201914,1379.05,1426.20,16741,34789,5,-29.35,24.35,-2.08,1.77
2,500002,201915,1408.60,1432.30,9008,21127,3,-12.80,23.70,-0.90,1.68
3,500002,201916,1433.05,1493.90,23703,50389,5,-16.40,55.40,-1.10,3.87
4,500002,201917,1473.40,1490.55,13102,27577,3,-4.10,16.65,-0.28,1.13


**Calculating HSDL, HSDG, HSDLp and HSDGp**

In [205]:
%%time
# Weekly change & Weekly change percent
df_temp2 = df_temp.copy()
df_temp2 = df_temp2.groupby(['sc_code', 'yearWeek'], sort=False).head(1)[['sc_code', 'yearWeek', 'close']].copy()
df_temp2.columns = ['sc_code', 'yearWeek', 'first']

df_temp3 = df_temp.copy()
df_temp3 = df_temp3.groupby(['sc_code', 'yearWeek'], sort=False).tail(1)[['sc_code', 'yearWeek', 'close']]
df_temp3.columns = ['sc_code', 'yearWeek', 'last']

df_temp4 = pd.merge(df_temp2, df_temp3, how='left', on=['sc_code', 'yearWeek'])

df_temp4['wChng'] = (df_temp4['last'] - df_temp4['first'])
df_temp4['wChngp'] = np.round( ( (df_temp4['last'] / df_temp4['first']) * 100 ) - 100, 2)

df_weekly = pd.merge(df_weekly, df_temp4, how='left', on=['sc_code', 'yearWeek'])

Wall time: 49 ms


**Capturing last traded day**

In [206]:
%%time
# lastTrdDoW - Last traded day of the week
df_temp3 = df_temp.copy()
df_temp3 = df_temp3.groupby(['sc_code', 'yearWeek'], sort=False).tail(1)[['sc_code', 'yearWeek', 'ts']]
df_temp3.columns = ['sc_code', 'yearWeek', 'lastTrdDoW']
df_weekly = pd.merge(df_weekly, df_temp3, how='left', on=['sc_code', 'yearWeek'])

Wall time: 24 ms


In [207]:
df_weekly.head()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,HSDL,HSDG,HSDLp,HSDGp,first,last,wChng,wChngp,lastTrdDoW
0,500002,201913,1313.55,1374.75,11210,27791,5,-6.30,48.75,-0.48,3.68,1313.55,1374.75,61.20,4.66,2019-04-05
1,500002,201914,1379.05,1426.20,16741,34789,5,-29.35,24.35,-2.08,1.77,1398.20,1426.20,28.00,2.00,2019-04-12
2,500002,201915,1408.60,1432.30,9008,21127,3,-12.80,23.70,-0.90,1.68,1421.40,1432.30,10.90,0.77,2019-04-18
3,500002,201916,1433.05,1493.90,23703,50389,5,-16.40,55.40,-1.10,3.87,1433.05,1477.50,44.45,3.10,2019-04-26
4,500002,201917,1473.40,1490.55,13102,27577,3,-4.10,16.65,-0.28,1.13,1473.40,1490.55,17.15,1.16,2019-05-03


In [208]:
df_weekly.tail()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,HSDL,HSDG,HSDLp,HSDGp,first,last,wChng,wChngp,lastTrdDoW
48,500002,202008,1184.55,1200.05,3284,5361,5,-36.55,7.70,-2.97,0.65,1192.35,1184.55,-7.80,-0.65,2020-02-28
49,500002,202009,1163.60,1193.90,2941,6282,5,-26.20,13.55,-2.20,1.15,1189.80,1193.90,4.10,0.34,2020-03-06
50,500002,202010,1011.30,1126.65,8438,14051,4,-71.70,-16.80,-6.46,-1.49,1126.65,1011.30,-115.35,-10.24,2020-03-13
51,500002,202011,893.00,964.65,5067,6816,5,-46.65,65.80,-4.61,7.37,964.65,958.80,-5.85,-0.61,2020-03-20
52,500002,202012,827.90,870.10,12738,47460,5,-88.70,24.60,-9.25,2.94,870.10,853.10,-17.00,-1.95,2020-03-27


**Calculating times increased(TI)**

In [209]:
%%time
# Sum up times increased in the week
df_temp2 = df_temp.copy()
df_temp2['TI'] = np.where(df_temp2['close'] > df_temp2['prevclose'], 1, 0)

df_temp2 = df_temp2.groupby(['sc_code', 'yearWeek'], sort=False)['TI'].agg([('TI','sum')]).reset_index().copy()
df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=['sc_code', 'yearWeek'])

Wall time: 57.6 ms


**Calculating highVol, avgVol(without high volume) and HVdAV**

In [210]:
%%time
# High Volumes / Average Volume = Will tell you how many times volumes was higher than average in that week
df_temp2 = df_temp.sort_values(['sc_code', 'yearWeek', 'volumes'], ascending=[True, True, False]).copy()

df_temp3 = df_temp2.groupby(['sc_code', 'yearWeek'], sort=False)['sc_code', 'yearWeek', 'volumes'].head(1)
df_temp3.columns = ['sc_code', 'yearWeek', 'highVol']

# this has a bug as tail(4) assumes every week has 5 days. When there are holidays, it might include high volumes as well.
df_temp4 = df_temp2.groupby(['sc_code', 'yearWeek'], sort=False)['sc_code', 'yearWeek', 'volumes'].tail(4) \
            .groupby(['sc_code', 'yearWeek'], sort=False)['volumes'].agg([('avgVol','mean')]).reset_index().copy()
df_temp4['avgVol'] = np.round(df_temp4['avgVol'],2)

df_temp5 = pd.merge(df_temp3, df_temp4, how='left', on=['sc_code', 'yearWeek'])
df_temp5['HVdAV'] = np.round(df_temp5['highVol'] / df_temp5['avgVol'],2)

df_temp5 = df_temp5[['sc_code', 'yearWeek', 'avgVol', 'HVdAV']]
# volume average without high volume data as that increases the average volume, decided to have this column
df_temp5.columns = ['sc_code', 'yearWeek', 'volAvgWOhv', 'HVdAV']

df_weekly = pd.merge(df_weekly, df_temp5, how='left', on=['sc_code', 'yearWeek'])

Wall time: 125 ms


**Calculating CPveoHVD**

In [211]:
%%time
# Closed +ve on volume high day
df_temp2 = df_temp.sort_values(['sc_code', 'yearWeek', 'volumes'], ascending=[True, True, False]).copy()

# SettingWithCopyWarning is issued when statement executed without .copy() 
# SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
df_temp2 = df_temp2.groupby(['sc_code', 'yearWeek'], sort=False).head(1).copy()

df_temp2['CPveoHVD'] = np.where(df_temp2['close'] > df_temp2['prevclose'], 1, 0)
df_temp2 = df_temp2[['sc_code', 'yearWeek', 'CPveoHVD']]

df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=['sc_code', 'yearWeek'])

Wall time: 61.9 ms


In [212]:
df_weekly.head(10)

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,HSDL,HSDG,HSDLp,HSDGp,first,last,wChng,wChngp,lastTrdDoW,TI,volAvgWOhv,HVdAV,CPveoHVD
0,500002,201913,1313.55,1374.75,11210,27791,5,-6.30,48.75,-0.48,3.68,1313.55,1374.75,61.20,4.66,2019-04-05,3,7065.00,3.93,1
1,500002,201914,1379.05,1426.20,16741,34789,5,-29.35,24.35,-2.08,1.77,1398.20,1426.20,28.00,2.00,2019-04-12,4,12229.25,2.84,1
2,500002,201915,1408.60,1432.30,9008,21127,3,-12.80,23.70,-0.90,1.68,1421.40,1432.30,10.90,0.77,2019-04-18,1,9008.00,2.35,1
3,500002,201916,1433.05,1493.90,23703,50389,5,-16.40,55.40,-1.10,3.87,1433.05,1477.50,44.45,3.10,2019-04-26,3,17032.00,2.96,1
4,500002,201917,1473.40,1490.55,13102,27577,3,-4.10,16.65,-0.28,1.13,1473.40,1490.55,17.15,1.16,2019-05-03,2,13102.67,2.10,0
5,500002,201918,1376.95,1481.30,19429,47397,5,-56.65,-9.25,-3.91,-0.62,1481.30,1376.95,-104.35,-7.04,2019-05-10,0,12437.00,3.81,0
6,500002,201919,1346.55,1392.05,7935,31284,5,-30.40,24.30,-2.21,1.80,1346.55,1392.05,45.50,3.38,2019-05-17,4,2098.75,14.91,1
7,500002,201920,1426.00,1494.65,10970,37266,5,-22.85,65.30,-1.57,4.57,1431.90,1494.65,62.75,4.38,2019-05-24,3,4397.00,8.48,1
8,500002,201921,1571.00,1610.70,32984,119315,5,-20.80,116.05,-1.29,7.76,1610.70,1571.00,-39.70,-2.46,2019-05-31,1,11401.25,10.47,0
9,500002,201922,1527.60,1591.85,10052,21913,4,-32.60,36.75,-2.09,2.41,1591.85,1564.35,-27.50,-1.73,2019-06-07,2,10052.50,2.18,0


In [213]:
df_weekly.tail(5)

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,HSDL,HSDG,HSDLp,HSDGp,first,last,wChng,wChngp,lastTrdDoW,TI,volAvgWOhv,HVdAV,CPveoHVD
48,500002,202008,1184.55,1200.05,3284,5361,5,-36.55,7.70,-2.97,0.65,1192.35,1184.55,-7.80,-0.65,2020-02-28,1,2765.25,1.94,0
49,500002,202009,1163.60,1193.90,2941,6282,5,-26.20,13.55,-2.20,1.15,1189.80,1193.90,4.10,0.34,2020-03-06,4,2106.00,2.98,1
50,500002,202010,1011.30,1126.65,8438,14051,4,-71.70,-16.80,-6.46,-1.49,1126.65,1011.30,-115.35,-10.24,2020-03-13,0,8438.50,1.67,0
51,500002,202011,893.00,964.65,5067,6816,5,-46.65,65.80,-4.61,7.37,964.65,958.80,-5.85,-0.61,2020-03-20,1,4630.75,1.47,0
52,500002,202012,827.90,870.10,12738,47460,5,-88.70,24.60,-9.25,2.94,870.10,853.10,-17.00,-1.95,2020-03-27,2,4058.25,11.69,0


**Calculating Last day volume of the week**

In [214]:
%%time
# Last day volume of the week
df_temp2 = df_temp.copy()
df_temp2 = df_temp2.groupby(['sc_code', 'yearWeek'], sort=False).tail(1)[['sc_code', 'yearWeek', 'volumes']]
df_temp2.columns = ['sc_code', 'yearWeek', 'lastDVotWk']

df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=['sc_code', 'yearWeek'])

Wall time: 49.2 ms


In [215]:
df_weekly['lastDVdAV'] = np.round(df_weekly['lastDVotWk'] / df_weekly['volAvgWOhv'],2)

In [333]:
df_weekly.head()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,HSDL,HSDG,HSDLp,HSDGp,first,last,wChng,wChngp,lastTrdDoW,TI,volAvgWOhv,HVdAV,CPveoHVD,lastDVotWk,lastDVdAV
0,500002,201913,1313.55,1374.75,11210,27791,5,-6.30,48.75,-0.48,3.68,1313.55,1374.75,61.20,4.66,2019-04-05,3,7065.00,3.93,1,12887,1.82
1,500002,201914,1379.05,1426.20,16741,34789,5,-29.35,24.35,-2.08,1.77,1398.20,1426.20,28.00,2.00,2019-04-12,4,12229.25,2.84,1,6928,0.57
2,500002,201915,1408.60,1432.30,9008,21127,3,-12.80,23.70,-0.90,1.68,1421.40,1432.30,10.90,0.77,2019-04-18,1,9008.00,2.35,1,21127,2.35
3,500002,201916,1433.05,1493.90,23703,50389,5,-16.40,55.40,-1.10,3.87,1433.05,1477.50,44.45,3.10,2019-04-26,3,17032.00,2.96,1,23219,1.36
4,500002,201917,1473.40,1490.55,13102,27577,3,-4.10,16.65,-0.28,1.13,1473.40,1490.55,17.15,1.16,2019-05-03,2,13102.67,2.10,0,4414,0.34


In [48]:
df_temp.tail()

,ts,sc_code,sc_name,high,low,close,prevclose,volumes,yearWeek,chng,chngp
644882,2020-03-23,500002,ABB LTD.,994.15,809.95,870.10,958.80,47460,202012,-88.70,-9.25
647299,2020-03-24,500002,ABB LTD.,890.70,824.10,827.90,870.10,3363,202012,-42.20,-4.85
649725,2020-03-25,500002,ABB LTD.,847.30,795.00,835.85,827.90,3813,202012,7.95,0.96
652096,2020-03-26,500002,ABB LTD.,870.35,835.00,860.45,835.85,3853,202012,24.60,2.94
654560,2020-03-27,500002,ABB LTD.,914.00,840.00,853.10,860.45,5204,202012,-7.35,-0.85


In [343]:
df_weekly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 22 columns):
sc_code       53 non-null int64
yearWeek      53 non-null int64
closeL        53 non-null float64
closeH        53 non-null float64
volAvg        53 non-null int32
volHigh       53 non-null int64
daysTraded    53 non-null int64
HSDL          53 non-null float64
HSDG          53 non-null float64
HSDLp         53 non-null float64
HSDGp         53 non-null float64
first         53 non-null float64
last          53 non-null float64
wChng         53 non-null float64
wChngp        53 non-null float64
lastTrdDoW    53 non-null datetime64[ns]
TI            53 non-null int32
volAvgWOhv    53 non-null float64
HVdAV         53 non-null float64
CPveoHVD      53 non-null int32
lastDVotWk    53 non-null int64
lastDVdAV     53 non-null float64
dtypes: datetime64[ns](1), float64(13), int32(3), int64(5)
memory usage: 8.9 KB


### Quality & Tidiness

##### Quality
No quality issues found in original dataset


##### Tidiness

1. Ignoring not used columns no_trades, net_turnover, tdcloindi, isin
1. Rename column NO_OF_SHRS to volume
1. Create a new dataframe and add below new columns, 
    + closeH - high close in the week
    + closeL - low closein the week
    + volHigh - Highest volume in the week
    + volAvg - Volume average
    + daysTraded - Number of days traded in the week
    + HSDL - Highest Single Day Loss
    + HSDG - Highest Single Day Gain
    + HSDLp - Highest Single Day Loss percent
    + HSDGp - Highest Single Day Gain percent
    + first - First close of the week
    + last - Last close of he week
    + wChng - Week change
    + wChngp - Week change percent
    + lastTrdDoW - Last traded day of week
    + TI - Times increased
    + volAvgWOhv - Volume average without high volume
    + HVdAV - High volume / Average volume(without highvolume)
    + CPveoHVD - Close positive on high volume day
    + lastDVotWk - Last day volume
    + lastDVdAV - Last day volume / average volume


### Cleaning

##### Functions

In [6]:
# all the above calculations are added here in this function
# Setting sort=False during groupby as "Groupby preserves the order of rows within each group"

def weekly_trend_analysis(ticker, df_weekly_all, df_daily):
    df_temp = df_daily.copy()

    # Adding column yearWeek
    df_temp['yearWeek'] = df_temp['ts'].dt.year * 100 + df_temp['ts'].dt.strftime('%U').astype(int)

    # Change & change percent
    df_temp['chng'] = (df_temp['close'] - df_temp['prevclose'])
    df_temp['chngp'] = np.round( ( (df_temp['close'] / df_temp['prevclose']) * 100 ) - 100, 2)
   
    # close(High/Low), volume average of all days in week, highest volume of the week
    df_temp2 = df_temp.groupby([ticker, 'yearWeek'], sort=False)['close'].agg([('L','min'),('H', 'max')]) \
        .add_prefix('close').reset_index().copy()
    df_temp3 = df_temp.groupby([ticker, 'yearWeek'], sort=False)['volumes'].agg([('Avg','mean'),('High', 'max')]) \
        .add_prefix('vol').reset_index().copy()
    df_temp3['volAvg']= df_temp3['volAvg'].astype(int)
    df_temp3 = pd.merge(df_temp2, df_temp3, how='left', on=[ticker, 'yearWeek'])
    #df_grouped = df_grouped.rename(columns={'close':'hClose'})
    df_weekly = df_temp3.copy()

    # Count number of trading days in the week
    df_temp2 = df_temp.groupby([ticker, 'yearWeek'], sort=False).count().reset_index()[[ticker, 'yearWeek', 'ts']]
    df_temp2.columns = [ticker, 'yearWeek', 'daysTraded']
    df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=[ticker, 'yearWeek'])    
    
    # lastTrdDoW - Last traded day of the week
    #df_temp3 = df_temp.copy()
    df_temp3 = df_temp.groupby([ticker, 'yearWeek'], sort=False).tail(1)[[ticker, 'yearWeek', 'ts']]
    df_temp3.columns = [ticker, 'yearWeek', 'lastTrdDoW']
    df_weekly = pd.merge(df_weekly, df_temp3, how='left', on=[ticker, 'yearWeek'])    
    
    # High Single Day chng/chngp
    df_temp2 = df_temp.groupby([ticker, 'yearWeek'], sort=False)['chng'].agg([('L','min'),('G', 'max')]) \
                .add_prefix('HSD').reset_index().copy()
    df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=[ticker, 'yearWeek'])

    df_temp2 = df_temp.groupby([ticker, 'yearWeek'], sort=False)['chngp'].agg([('Lp','min'),('Gp', 'max')]) \
                .add_prefix('HSD').reset_index().copy()
    df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=[ticker, 'yearWeek'])

    # Weekly change & Weekly change percent
    df_temp2 = df_temp.copy()
    df_temp2 = df_temp2.groupby([ticker, 'yearWeek'], sort=False).head(1)[[ticker, 'yearWeek', 'close']].copy()
    df_temp2.columns = [ticker, 'yearWeek', 'first']

    df_temp3 = df_temp.copy()
    df_temp3 = df_temp3.groupby([ticker, 'yearWeek'], sort=False).tail(1)[[ticker, 'yearWeek', 'close']]
    df_temp3.columns = [ticker, 'yearWeek', 'last']

    df_temp4 = pd.merge(df_temp2, df_temp3, how='left', on=[ticker, 'yearWeek'])

    df_temp4['wChng'] = (df_temp4['last'] - df_temp4['first'])
    df_temp4['wChngp'] = np.round( ( (df_temp4['last'] / df_temp4['first']) * 100 ) - 100, 2)

    df_weekly = pd.merge(df_weekly, df_temp4, how='left', on=[ticker, 'yearWeek'])

    # Sum up times increased in the week
    df_temp2 = df_temp.copy()
    df_temp2['TI'] = np.where(df_temp2['close'] > df_temp2['prevclose'], 1, 0)

    df_temp2 = df_temp2.groupby([ticker, 'yearWeek'], sort=False)['TI'].agg([('TI','sum')]).reset_index().copy()
    df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=[ticker, 'yearWeek'])

    # High Volumes / Average Volume = Will tell you how many times volumes was higher than average in that week
    df_temp2 = df_temp.sort_values([ticker, 'yearWeek', 'volumes'], ascending=[True, True, False]).copy()

    df_temp3 = df_temp2.groupby([ticker, 'yearWeek'], sort=False)[ticker, 'yearWeek', 'volumes'].head(1)
    df_temp3.columns = [ticker, 'yearWeek', 'highVol']

    df_temp4 = df_temp2.groupby([ticker, 'yearWeek'], sort=False)[ticker, 'yearWeek', 'volumes'].tail(4) \
                .groupby([ticker, 'yearWeek'], sort=False)['volumes'].agg([('avgVol','mean')]).reset_index().copy()
    df_temp4['avgVol'] = np.round(df_temp4['avgVol'],2)

    df_temp5 = pd.merge(df_temp3, df_temp4, how='left', on=[ticker, 'yearWeek'])
    df_temp5['HVdAV'] = np.round(df_temp5['highVol'] / df_temp5['avgVol'],2)

    df_temp5 = df_temp5[[ticker, 'yearWeek', 'avgVol', 'HVdAV']]
    # volume average without high volume data as that increases the average volume, decided to have this column
    df_temp5.columns = [ticker, 'yearWeek', 'volAvgWOhv', 'HVdAV']

    df_weekly = pd.merge(df_weekly, df_temp5, how='left', on=[ticker, 'yearWeek'])

    # Closed +ve on volume high day
    df_temp2 = df_temp.sort_values([ticker, 'yearWeek', 'volumes'], ascending=[True, True, False]).copy()

    # SettingWithCopyWarning is issued when statement executed without .copy() 
    # SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
    df_temp2 = df_temp2.groupby([ticker, 'yearWeek'], sort=False).head(1).copy()

    df_temp2['CPveoHVD'] = np.where(df_temp2['close'] > df_temp2['prevclose'], 1, 0)
    df_temp2 = df_temp2[[ticker, 'yearWeek', 'CPveoHVD']]

    df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=[ticker, 'yearWeek'])

    # Last/Latest day volume of the week. Later used only for latest week's last day volume
    df_temp2 = df_temp.copy()
    df_temp2 = df_temp2.groupby([ticker, 'yearWeek'], sort=False).tail(1)[[ticker, 'yearWeek', 'volumes']]
    df_temp2.columns = [ticker, 'yearWeek', 'lastDVotWk']

    df_weekly = pd.merge(df_weekly, df_temp2, how='left', on=[ticker, 'yearWeek'])
    df_weekly['lastDVdAV'] = np.round(df_weekly['lastDVotWk'] / df_weekly['volAvgWOhv'],2)

    # df_weekly_all will be 0, when its a new company or its a FTA(First Time Analysis)
    if df_weekly_all.shape[0] == 0:
        df_weekly_all = pd.DataFrame(columns=list(df_weekly.columns))       
        
    # Removing all yearWeek in df_weekly2 from df_weekly
    a = set(df_weekly_all['yearWeek'])
    b = set(df_weekly['yearWeek'])
    c = list(a.difference(b))
    #print('df_weekly_all={}, df_weekly={}, difference={}'.format(len(a), len(b), len(c)) )
    df_weekly_all = df_weekly_all[df_weekly_all.yearWeek.isin(c)].copy()

    # Append the latest week data to df_weekly
    df_weekly_all = pd.concat([df_weekly_all, df_weekly], sort=False)
    #print('After concat : df_weekly_all={}'.format(df_weekly_all.shape[0]))    
        
    return df_weekly_all



#### Stage 1 - Single company execution

Initilizations

In [11]:
#df_weekly = df_weekly[0:0]
df_weekly = pd.DataFrame() 

In [12]:
df_weekly.columns

Index([], dtype='object')

Refresh the dataframe & retry

In [13]:
df_bse_daily.columns

Index(['ts', 'sc_code', 'sc_name', 'sc_group', 'sc_type', 'open', 'high',
       'low', 'close', 'last', 'prevclose', 'no_trades', 'no_of_shrs',
       'net_turnover', 'tdcloindi', 'isin'],
      dtype='object')

In [14]:
df_bse_bk = df_bse_daily.copy()

In [ ]:
df_bse_daily = df_bse_bk.copy()

In [20]:
%%time
exchange = 'BSE'
sc_code = 500002 #ABB
#df_weekly_all = pd.DataFrame() 

ticker = 'sc_code'
df_daily = df_bse_daily[df_bse_daily.sc_code == sc_code][['ts', 'sc_code', 'sc_name', 'high', 'low', 'close', 'prevclose', 'no_of_shrs']]
df_daily.rename(columns={"no_of_shrs": "volumes"}, inplace=True)

# FTA = First Time Analysis
df_weekly2 = weekly_trend_analysis(ticker, df_weekly, df_daily)

Wall time: 301 ms


In [21]:
df_weekly2.head()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,lastTrdDoW,HSDL,HSDG,HSDLp,HSDGp,first,last,wChng,wChngp,TI,volAvgWOhv,HVdAV,CPveoHVD,lastDVotWk,lastDVdAV
0,500002,201913,1313.55,1374.75,11210,27791,5,2019-04-05,-6.30,48.75,-0.48,3.68,1313.55,1374.75,61.20,4.66,3,7065.00,3.93,1,12887,1.82
1,500002,201914,1379.05,1426.20,16741,34789,5,2019-04-12,-29.35,24.35,-2.08,1.77,1398.20,1426.20,28.00,2.00,4,12229.25,2.84,1,6928,0.57
2,500002,201915,1408.60,1432.30,9008,21127,3,2019-04-18,-12.80,23.70,-0.90,1.68,1421.40,1432.30,10.90,0.77,1,9008.00,2.35,1,21127,2.35
3,500002,201916,1433.05,1493.90,23703,50389,5,2019-04-26,-16.40,55.40,-1.10,3.87,1433.05,1477.50,44.45,3.10,3,17032.00,2.96,1,23219,1.36
4,500002,201917,1473.40,1490.55,13102,27577,3,2019-05-03,-4.10,16.65,-0.28,1.13,1473.40,1490.55,17.15,1.16,2,13102.67,2.10,0,4414,0.34


In [22]:
df_weekly2.tail()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh,daysTraded,lastTrdDoW,HSDL,HSDG,HSDLp,HSDGp,first,last,wChng,wChngp,TI,volAvgWOhv,HVdAV,CPveoHVD,lastDVotWk,lastDVdAV
48,500002,202008,1184.55,1200.05,3284,5361,5,2020-02-28,-36.55,7.70,-2.97,0.65,1192.35,1184.55,-7.80,-0.65,1,2765.25,1.94,0,5361,1.94
49,500002,202009,1163.60,1193.90,2941,6282,5,2020-03-06,-26.20,13.55,-2.20,1.15,1189.80,1193.90,4.10,0.34,4,2106.00,2.98,1,1955,0.93
50,500002,202010,1011.30,1126.65,8438,14051,4,2020-03-13,-71.70,-16.80,-6.46,-1.49,1126.65,1011.30,-115.35,-10.24,0,8438.50,1.67,0,14051,1.67
51,500002,202011,893.00,964.65,5067,6816,5,2020-03-20,-46.65,65.80,-4.61,7.37,964.65,958.80,-5.85,-0.61,1,4630.75,1.47,0,5207,1.12
52,500002,202012,827.90,870.10,12738,47460,5,2020-03-27,-88.70,24.60,-9.25,2.94,870.10,853.10,-17.00,-1.95,2,4058.25,11.69,0,5204,1.28


In [27]:
df_daily.tail()

,ts,sc_code,sc_name,high,low,close,prevclose,volumes
644882,2020-03-23,500002,ABB LTD.,994.15,809.95,870.10,958.80,47460
647299,2020-03-24,500002,ABB LTD.,890.70,824.10,827.90,870.10,3363
649725,2020-03-25,500002,ABB LTD.,847.30,795.00,835.85,827.90,3813
652096,2020-03-26,500002,ABB LTD.,870.35,835.00,860.45,835.85,3853
654560,2020-03-27,500002,ABB LTD.,914.00,840.00,853.10,860.45,5204


### Step 3 : Define the data model

In [23]:
df_weekly2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 22 columns):
sc_code       53 non-null object
yearWeek      53 non-null object
closeL        53 non-null float64
closeH        53 non-null float64
volAvg        53 non-null object
volHigh       53 non-null object
daysTraded    53 non-null object
lastTrdDoW    53 non-null datetime64[ns]
HSDL          53 non-null float64
HSDG          53 non-null float64
HSDLp         53 non-null float64
HSDGp         53 non-null float64
first         53 non-null float64
last          53 non-null float64
wChng         53 non-null float64
wChngp        53 non-null float64
TI            53 non-null object
volAvgWOhv    53 non-null float64
HVdAV         53 non-null float64
CPveoHVD      53 non-null object
lastDVotWk    53 non-null object
lastDVdAV     53 non-null float64
dtypes: datetime64[ns](1), float64(13), object(8)
memory usage: 7.9+ KB


### Step 4 : Run ETL

#### Stage 2 - For all company execution

In [36]:
#df_weekly = df_weekly[0:0]
df_weekly_all = pd.DataFrame() 

In [35]:
df_weekly_all.columns

Index([], dtype='object')

In [20]:
df_bse_bk = df_bse_daily.copy()

In [ ]:
df_bse_daily = df_bse_bk.copy()

In [154]:
%%time
# First Run - For full BSE run it took Wall time: 1h 26min 31s
exchange = 'BSE'
sc_code = 500002 #ABB
#sc_code = 532370 #Ramco Systems
ticker_col = 'sc_code'
bse_csv_cols = ['ts', 'sc_code', 'sc_name', 'high', 'low', 'close', 'prevclose', 'no_of_shrs']

ticker_list = df_bse_daily['sc_code'].unique()
print('Total sc_code = {} to process'.format(len(ticker_list)))

for ticker in ticker_list:
    #print(ticker)
    df_weekly = pd.DataFrame() 
    
    if not df_weekly_all.empty:
        df_weekly = df_weekly_all[df_weekly_all.sc_code == ticker].copy()
    
    # It will print for every company for first run
    if df_weekly.empty:
        print('Weekly empty for {}'.format(ticker))
    
    df_daily = df_bse_daily[df_bse_daily.sc_code == ticker][bse_csv_cols]
    #df_daily = df_bse_5daily[df_bse_5daily.sc_code == sc_code][['ts', 'sc_code', 'sc_name', 'high', 'low', 'close', 'prevclose', 'no_of_shrs']]
    df_daily.rename(columns={"no_of_shrs": "volumes"}, inplace=True)
    
    df_temp = weekly_trend_analysis(ticker_col, df_weekly, df_daily)
    
    # Only incase if df_weekly is empty. Happen during FTA(First Time Analysis)
    if df_weekly_all.shape[0] == 0:
        df_weekly_all = pd.DataFrame(columns=list(df_temp.columns))       
    
    df_weekly_all = pd.concat([df_weekly_all, df_temp], sort=False)


Total sc_code = 4298 to process
Weekly empty for 500002
Weekly empty for 500003
Weekly empty for 500008
Weekly empty for 500009
Weekly empty for 500010
Weekly empty for 500012
Weekly empty for 500013
Weekly empty for 500014
Weekly empty for 500016
Weekly empty for 500020
Weekly empty for 500023
Weekly empty for 500024
Weekly empty for 500027
Weekly empty for 500028
Weekly empty for 500029
Weekly empty for 500031
Weekly empty for 500032
Weekly empty for 500033
Weekly empty for 500034
Weekly empty for 500038
Weekly empty for 500039
Weekly empty for 500040
Weekly empty for 500041
Weekly empty for 500042
Weekly empty for 500043
Weekly empty for 500048
Weekly empty for 500049
Weekly empty for 500052
Weekly empty for 500055
Weekly empty for 500058
Weekly empty for 500059
Weekly empty for 500060
Weekly empty for 500067
Weekly empty for 500068
Weekly empty for 500069
Weekly empty for 500074
Weekly empty for 500078
Weekly empty for 500083
Weekly empty for 500084
Weekly empty for 500085
Weekly e

Weekly empty for 504028
Weekly empty for 504036
Weekly empty for 504058
Weekly empty for 504067
Weekly empty for 504076
Weekly empty for 504084
Weekly empty for 504092
Weekly empty for 504093
Weekly empty for 504112
Weekly empty for 504132
Weekly empty for 504176
Weekly empty for 504212
Weekly empty for 504220
Weekly empty for 504240
Weekly empty for 504258
Weekly empty for 504286
Weekly empty for 504335
Weekly empty for 504341
Weekly empty for 504614
Weekly empty for 504646
Weekly empty for 504741
Weekly empty for 504786
Weekly empty for 504840
Weekly empty for 504879
Weekly empty for 504908
Weekly empty for 504918
Weekly empty for 504959
Weekly empty for 504961
Weekly empty for 504966
Weekly empty for 504973
Weekly empty for 504988
Weekly empty for 505010
Weekly empty for 505029
Weekly empty for 505036
Weekly empty for 505075
Weekly empty for 505141
Weekly empty for 505160
Weekly empty for 505163
Weekly empty for 505192
Weekly empty for 505196
Weekly empty for 505200
Weekly empty for

Weekly empty for 512597
Weekly empty for 512599
Weekly empty for 512608
Weekly empty for 512634
Weekly empty for 513005
Weekly empty for 513010
Weekly empty for 513023
Weekly empty for 513043
Weekly empty for 513059
Weekly empty for 513063
Weekly empty for 513097
Weekly empty for 513108
Weekly empty for 513117
Weekly empty for 513119
Weekly empty for 513121
Weekly empty for 513142
Weekly empty for 513151
Weekly empty for 513179
Weekly empty for 513216
Weekly empty for 513228
Weekly empty for 513250
Weekly empty for 513252
Weekly empty for 513262
Weekly empty for 513269
Weekly empty for 513291
Weekly empty for 513305
Weekly empty for 513335
Weekly empty for 513349
Weekly empty for 513353
Weekly empty for 513359
Weekly empty for 513361
Weekly empty for 513369
Weekly empty for 513375
Weekly empty for 513377
Weekly empty for 513401
Weekly empty for 513414
Weekly empty for 513434
Weekly empty for 513436
Weekly empty for 513446
Weekly empty for 513452
Weekly empty for 513456
Weekly empty for

Weekly empty for 523323
Weekly empty for 523329
Weekly empty for 523367
Weekly empty for 523369
Weekly empty for 523371
Weekly empty for 523384
Weekly empty for 523385
Weekly empty for 523387
Weekly empty for 523391
Weekly empty for 523395
Weekly empty for 523398
Weekly empty for 523405
Weekly empty for 523411
Weekly empty for 523445
Weekly empty for 523449
Weekly empty for 523457
Weekly empty for 523465
Weekly empty for 523467
Weekly empty for 523475
Weekly empty for 523483
Weekly empty for 523489
Weekly empty for 523523
Weekly empty for 523537
Weekly empty for 523539
Weekly empty for 523550
Weekly empty for 523566
Weekly empty for 523574
Weekly empty for 523586
Weekly empty for 523598
Weekly empty for 523606
Weekly empty for 523610
Weekly empty for 523618
Weekly empty for 523620
Weekly empty for 523628
Weekly empty for 523630
Weekly empty for 523638
Weekly empty for 523642
Weekly empty for 523648
Weekly empty for 523660
Weekly empty for 523672
Weekly empty for 523694
Weekly empty for

Weekly empty for 530579
Weekly empty for 530585
Weekly empty for 530589
Weekly empty for 530609
Weekly empty for 530617
Weekly empty for 530621
Weekly empty for 530627
Weekly empty for 530643
Weekly empty for 530655
Weekly empty for 530665
Weekly empty for 530677
Weekly empty for 530689
Weekly empty for 530695
Weekly empty for 530697
Weekly empty for 530721
Weekly empty for 530733
Weekly empty for 530759
Weekly empty for 530773
Weekly empty for 530789
Weekly empty for 530803
Weekly empty for 530813
Weekly empty for 530815
Weekly empty for 530821
Weekly empty for 530825
Weekly empty for 530829
Weekly empty for 530843
Weekly empty for 530845
Weekly empty for 530853
Weekly empty for 530859
Weekly empty for 530867
Weekly empty for 530871
Weekly empty for 530879
Weekly empty for 530883
Weekly empty for 530885
Weekly empty for 530889
Weekly empty for 530897
Weekly empty for 530919
Weekly empty for 530931
Weekly empty for 530943
Weekly empty for 530951
Weekly empty for 530959
Weekly empty for

...

Weekly empty for 542503
Weekly empty for 542579
Weekly empty for 542580
Weekly empty for 542592
Weekly empty for 542597
Weekly empty for 542599
Weekly empty for 542602
Weekly empty for 555555
Weekly empty for 570001
Weekly empty for 570004
Weekly empty for 570005
Weekly empty for 590003
Weekly empty for 590005
Weekly empty for 590006
Weekly empty for 590013
Weekly empty for 590018
Weekly empty for 590021
Weekly empty for 590022
Weekly empty for 590024
Weekly empty for 590025
Weekly empty for 590030
Weekly empty for 590031
Weekly empty for 590041
Weekly empty for 590043
Weekly empty for 590051
Weekly empty for 590056
Weekly empty for 590057
Weekly empty for 590062
Weekly empty for 590065
Weekly empty for 590066
Weekly empty for 590068
Weekly empty for 590070
Weekly empty for 590071
Weekly empty for 590072
Weekly empty for 590073
Weekly empty for 590075
Weekly empty for 590078
Weekly empty for 590086
Weekly empty for 590095
Weekly empty for 590096
Weekly empty for 590097
Weekly empty for

Weekly empty for 530799
Weekly empty for 530807
Weekly empty for 530953
Weekly empty for 530997
Weekly empty for 531043
Weekly empty for 531091
Weekly empty for 531176
Weekly empty for 531206
Weekly empty for 531280
Weekly empty for 531360
Weekly empty for 531420
Weekly empty for 531509
Weekly empty for 531525
Weekly empty for 531574
Weekly empty for 531609
Weekly empty for 531692
Weekly empty for 531762
Weekly empty for 531846
Weekly empty for 531881
Weekly empty for 531909
Weekly empty for 531989
Weekly empty for 532011
Weekly empty for 532145
Weekly empty for 532344
Weekly empty for 532441
Weekly empty for 532690
Weekly empty for 532788
Weekly empty for 532806
Weekly empty for 532887
Weekly empty for 533298
Weekly empty for 533407
Weekly empty for 533411
Weekly empty for 533941
Weekly empty for 534618
Weekly empty for 534659
Weekly empty for 534796
Weekly empty for 535204
Weekly empty for 535730
Weekly empty for 538081
Weekly empty for 538092
Weekly empty for 538119
Weekly empty for

Weekly empty for 540733
Weekly empty for 540981
Weekly empty for 541805
Weekly empty for 501261
Weekly empty for 502901
Weekly empty for 503127
Weekly empty for 507522
Weekly empty for 513502
Weekly empty for 516020
Weekly empty for 524590
Weekly empty for 526009
Weekly empty for 526441
Weekly empty for 526506
Weekly empty for 530093
Weekly empty for 530263
Weekly empty for 530469
Weekly empty for 530533
Weekly empty for 530715
Weekly empty for 530723
Weekly empty for 531257
Weekly empty for 531370
Weekly empty for 531396
Weekly empty for 531416
Weekly empty for 531658
Weekly empty for 531878
Weekly empty for 532316
Weekly empty for 532403
Weekly empty for 532416
Weekly empty for 532866
Weekly empty for 538446
Weekly empty for 538608
Weekly empty for 538943
Weekly empty for 538993
Weekly empty for 539112
Weekly empty for 539222
Weekly empty for 539266
Weekly empty for 539470
Weekly empty for 540204
Weekly empty for 540393
Weekly empty for 540416
Weekly empty for 541252
Weekly empty for

Weekly empty for 519331
Weekly empty for 521036
Weekly empty for 531051
Weekly empty for 538465
Weekly empty for 540552
Weekly empty for 542670
Weekly empty for 500421
Weekly empty for 502294
Weekly empty for 502893
Weekly empty for 507946
Weekly empty for 509053
Weekly empty for 511187
Weekly empty for 511276
Weekly empty for 512233
Weekly empty for 512329
Weekly empty for 512441
Weekly empty for 513579
Weekly empty for 522036
Weekly empty for 524514
Weekly empty for 526473
Weekly empty for 530495
Weekly empty for 530899
Weekly empty for 531274
Weekly empty for 531553
Weekly empty for 531652
Weekly empty for 532057
Weekly empty for 536456
Weekly empty for 538451
Weekly empty for 538569
Weekly empty for 538862
Weekly empty for 539378
Weekly empty for 539433
Weekly empty for 540132
Weekly empty for 542669
Weekly empty for 538894
Weekly empty for 540079
Weekly empty for 540718
Weekly empty for 531775
Weekly empty for 535217
Weekly empty for 539117
Weekly empty for 526067
Weekly empty for

Weekly empty for 514414
Weekly empty for 524642
Weekly empty for 530161
Weekly empty for 530231
Weekly empty for 538857
Weekly empty for 542802
Weekly empty for 531506
Weekly empty for 542782
Weekly empty for 542823
Weekly empty for 542843
Weekly empty for 521048
Weekly empty for 542830
Weekly empty for 530705
Weekly empty for 539115
Weekly empty for 539288
Weekly empty for 542819
Weekly empty for 542839
Weekly empty for 542850
Weekly empty for 542851
Weekly empty for 542852
Weekly empty for 527005
Weekly empty for 531479
Weekly empty for 531887
Weekly empty for 538563
Weekly empty for 539114
Weekly empty for 542808
Weekly empty for 542812
Weekly empty for 506313
Weekly empty for 523351
Weekly empty for 538926
Weekly empty for 539091
Weekly empty for 532993
Weekly empty for 539116
Weekly empty for 542646
Weekly empty for 530909
Weekly empty for 542857
Weekly empty for 502850
Weekly empty for 531260
Weekly empty for 540140
Weekly empty for 542846
Weekly empty for 542862
Weekly empty for

In [37]:
%%time
# Second run - For full BSE run it took Wall time: 1h 41min 41s
exchange = 'BSE'
sc_code = 500002 #ABB
#sc_code = 532370 #Ramco Systems
ticker_col = 'sc_code'
bse_csv_cols = ['ts', 'sc_code', 'sc_name', 'high', 'low', 'close', 'prevclose', 'no_of_shrs']

ticker_list = df_bse_daily['sc_code'].unique()
print('Total sc_code = {} to process'.format(len(ticker_list)))

for ticker in ticker_list:
    #print(ticker)
    df_weekly = pd.DataFrame() 
    
    if not df_weekly_all.empty:
        df_weekly = df_weekly_all[df_weekly_all.sc_code == ticker].copy()
    
    # It will print for every company for first run
    if df_weekly.empty:
        print('Weekly empty for {}'.format(ticker))
    
    df_daily = df_bse_daily[df_bse_daily.sc_code == ticker][bse_csv_cols]
    #df_daily = df_bse_5daily[df_bse_5daily.sc_code == sc_code][['ts', 'sc_code', 'sc_name', 'high', 'low', 'close', 'prevclose', 'no_of_shrs']]
    df_daily.rename(columns={"no_of_shrs": "volumes"}, inplace=True)
    
    df_temp = weekly_trend_analysis(ticker_col, df_weekly, df_daily)
    
    # Only incase if df_weekly is empty. Happen during FTA(First Time Analysis)
    if df_weekly_all.shape[0] == 0:
        df_weekly_all = pd.DataFrame(columns=list(df_temp.columns))       
    
    df_weekly_all = pd.concat([df_weekly_all, df_temp], sort=False)


Total sc_code = 4298 to process
Weekly empty for 500002
Weekly empty for 500003
Weekly empty for 500008
Weekly empty for 500009
Weekly empty for 500010
Weekly empty for 500012
Weekly empty for 500013
Weekly empty for 500014
Weekly empty for 500016
Weekly empty for 500020
Weekly empty for 500023
Weekly empty for 500024
Weekly empty for 500027
Weekly empty for 500028
Weekly empty for 500029
Weekly empty for 500031
Weekly empty for 500032
Weekly empty for 500033
Weekly empty for 500034
Weekly empty for 500038
Weekly empty for 500039
Weekly empty for 500040
Weekly empty for 500041
Weekly empty for 500042
Weekly empty for 500043
Weekly empty for 500048
Weekly empty for 500049
Weekly empty for 500052
Weekly empty for 500055
Weekly empty for 500058
Weekly empty for 500059
Weekly empty for 500060
Weekly empty for 500067
Weekly empty for 500068
Weekly empty for 500069
Weekly empty for 500074
Weekly empty for 500078
Weekly empty for 500083
Weekly empty for 500084
Weekly empty for 500085
Weekly e

Weekly empty for 504028
Weekly empty for 504036
Weekly empty for 504058
Weekly empty for 504067
Weekly empty for 504076
Weekly empty for 504084
Weekly empty for 504092
Weekly empty for 504093
Weekly empty for 504112
Weekly empty for 504132
Weekly empty for 504176
Weekly empty for 504212
Weekly empty for 504220
Weekly empty for 504240
Weekly empty for 504258
Weekly empty for 504286
Weekly empty for 504335
Weekly empty for 504341
Weekly empty for 504614
Weekly empty for 504646
Weekly empty for 504741
Weekly empty for 504786
Weekly empty for 504840
Weekly empty for 504879
Weekly empty for 504908
Weekly empty for 504918
Weekly empty for 504959
Weekly empty for 504961
Weekly empty for 504966
Weekly empty for 504973
Weekly empty for 504988
Weekly empty for 505010
Weekly empty for 505029
Weekly empty for 505036
Weekly empty for 505075
Weekly empty for 505141
Weekly empty for 505160
Weekly empty for 505163
Weekly empty for 505192
Weekly empty for 505196
Weekly empty for 505200
Weekly empty for

Weekly empty for 512597
Weekly empty for 512599
Weekly empty for 512608
Weekly empty for 512634
Weekly empty for 513005
Weekly empty for 513010
Weekly empty for 513023
Weekly empty for 513043
Weekly empty for 513059
Weekly empty for 513063
Weekly empty for 513097
Weekly empty for 513108
Weekly empty for 513117
Weekly empty for 513119
Weekly empty for 513121
Weekly empty for 513142
Weekly empty for 513151
Weekly empty for 513179
Weekly empty for 513216
Weekly empty for 513228
Weekly empty for 513250
Weekly empty for 513252
Weekly empty for 513262
Weekly empty for 513269
Weekly empty for 513291
Weekly empty for 513305
Weekly empty for 513335
Weekly empty for 513349
Weekly empty for 513353
Weekly empty for 513359
Weekly empty for 513361
Weekly empty for 513369
Weekly empty for 513375
Weekly empty for 513377
Weekly empty for 513401
Weekly empty for 513414
Weekly empty for 513434
Weekly empty for 513436
Weekly empty for 513446
Weekly empty for 513452
Weekly empty for 513456
Weekly empty for

Weekly empty for 523323
Weekly empty for 523329
Weekly empty for 523367
Weekly empty for 523369
Weekly empty for 523371
Weekly empty for 523384
Weekly empty for 523385
Weekly empty for 523387
Weekly empty for 523391
Weekly empty for 523395
Weekly empty for 523398
Weekly empty for 523405
Weekly empty for 523411
Weekly empty for 523445
Weekly empty for 523449
Weekly empty for 523457
Weekly empty for 523465
Weekly empty for 523467
Weekly empty for 523475
Weekly empty for 523483
Weekly empty for 523489
Weekly empty for 523523
Weekly empty for 523537
Weekly empty for 523539
Weekly empty for 523550
Weekly empty for 523566
Weekly empty for 523574
Weekly empty for 523586
Weekly empty for 523598
Weekly empty for 523606
Weekly empty for 523610
Weekly empty for 523618
Weekly empty for 523620
Weekly empty for 523628
Weekly empty for 523630
Weekly empty for 523638
Weekly empty for 523642
Weekly empty for 523648
Weekly empty for 523660
Weekly empty for 523672
Weekly empty for 523694
Weekly empty for

Weekly empty for 530579
Weekly empty for 530585
Weekly empty for 530589
Weekly empty for 530609
Weekly empty for 530617
Weekly empty for 530621
Weekly empty for 530627
Weekly empty for 530643
Weekly empty for 530655
Weekly empty for 530665
Weekly empty for 530677
Weekly empty for 530689
Weekly empty for 530695
Weekly empty for 530697
Weekly empty for 530721
Weekly empty for 530733
Weekly empty for 530759
Weekly empty for 530773
Weekly empty for 530789
Weekly empty for 530803
Weekly empty for 530813
Weekly empty for 530815
Weekly empty for 530821
Weekly empty for 530825
Weekly empty for 530829
Weekly empty for 530843
Weekly empty for 530845
Weekly empty for 530853
Weekly empty for 530859
Weekly empty for 530867
Weekly empty for 530871
Weekly empty for 530879
Weekly empty for 530883
Weekly empty for 530885
Weekly empty for 530889
Weekly empty for 530897
Weekly empty for 530919
Weekly empty for 530931
Weekly empty for 530943
Weekly empty for 530951
Weekly empty for 530959
Weekly empty for

...

Weekly empty for 542503
Weekly empty for 542579
Weekly empty for 542580
Weekly empty for 542592
Weekly empty for 542597
Weekly empty for 542599
Weekly empty for 542602
Weekly empty for 555555
Weekly empty for 570001
Weekly empty for 570004
Weekly empty for 570005
Weekly empty for 590003
Weekly empty for 590005
Weekly empty for 590006
Weekly empty for 590013
Weekly empty for 590018
Weekly empty for 590021
Weekly empty for 590022
Weekly empty for 590024
Weekly empty for 590025
Weekly empty for 590030
Weekly empty for 590031
Weekly empty for 590041
Weekly empty for 590043
Weekly empty for 590051
Weekly empty for 590056
Weekly empty for 590057
Weekly empty for 590062
Weekly empty for 590065
Weekly empty for 590066
Weekly empty for 590068
Weekly empty for 590070
Weekly empty for 590071
Weekly empty for 590072
Weekly empty for 590073
Weekly empty for 590075
Weekly empty for 590078
Weekly empty for 590086
Weekly empty for 590095
Weekly empty for 590096
Weekly empty for 590097
Weekly empty for

Weekly empty for 530799
Weekly empty for 530807
Weekly empty for 530953
Weekly empty for 530997
Weekly empty for 531043
Weekly empty for 531091
Weekly empty for 531176
Weekly empty for 531206
Weekly empty for 531280
Weekly empty for 531360
Weekly empty for 531420
Weekly empty for 531509
Weekly empty for 531525
Weekly empty for 531574
Weekly empty for 531609
Weekly empty for 531692
Weekly empty for 531762
Weekly empty for 531846
Weekly empty for 531881
Weekly empty for 531909
Weekly empty for 531989
Weekly empty for 532011
Weekly empty for 532145
Weekly empty for 532344
Weekly empty for 532441
Weekly empty for 532690
Weekly empty for 532788
Weekly empty for 532806
Weekly empty for 532887
Weekly empty for 533298
Weekly empty for 533407
Weekly empty for 533411
Weekly empty for 533941
Weekly empty for 534618
Weekly empty for 534659
Weekly empty for 534796
Weekly empty for 535204
Weekly empty for 535730
Weekly empty for 538081
Weekly empty for 538092
Weekly empty for 538119
Weekly empty for

Weekly empty for 540733
Weekly empty for 540981
Weekly empty for 541805
Weekly empty for 501261
Weekly empty for 502901
Weekly empty for 503127
Weekly empty for 507522
Weekly empty for 513502
Weekly empty for 516020
Weekly empty for 524590
Weekly empty for 526009
Weekly empty for 526441
Weekly empty for 526506
Weekly empty for 530093
Weekly empty for 530263
Weekly empty for 530469
Weekly empty for 530533
Weekly empty for 530715
Weekly empty for 530723
Weekly empty for 531257
Weekly empty for 531370
Weekly empty for 531396
Weekly empty for 531416
Weekly empty for 531658
Weekly empty for 531878
Weekly empty for 532316
Weekly empty for 532403
Weekly empty for 532416
Weekly empty for 532866
Weekly empty for 538446
Weekly empty for 538608
Weekly empty for 538943
Weekly empty for 538993
Weekly empty for 539112
Weekly empty for 539222
Weekly empty for 539266
Weekly empty for 539470
Weekly empty for 540204
Weekly empty for 540393
Weekly empty for 540416
Weekly empty for 541252
Weekly empty for

Weekly empty for 519331
Weekly empty for 521036
Weekly empty for 531051
Weekly empty for 538465
Weekly empty for 540552
Weekly empty for 542670
Weekly empty for 500421
Weekly empty for 502294
Weekly empty for 502893
Weekly empty for 507946
Weekly empty for 509053
Weekly empty for 511187
Weekly empty for 511276
Weekly empty for 512233
Weekly empty for 512329
Weekly empty for 512441
Weekly empty for 513579
Weekly empty for 522036
Weekly empty for 524514
Weekly empty for 526473
Weekly empty for 530495
Weekly empty for 530899
Weekly empty for 531274
Weekly empty for 531553
Weekly empty for 531652
Weekly empty for 532057
Weekly empty for 536456
Weekly empty for 538451
Weekly empty for 538569
Weekly empty for 538862
Weekly empty for 539378
Weekly empty for 539433
Weekly empty for 540132
Weekly empty for 542669
Weekly empty for 538894
Weekly empty for 540079
Weekly empty for 540718
Weekly empty for 531775
Weekly empty for 535217
Weekly empty for 539117
Weekly empty for 526067
Weekly empty for

Weekly empty for 514414
Weekly empty for 524642
Weekly empty for 530161
Weekly empty for 530231
Weekly empty for 538857
Weekly empty for 542802
Weekly empty for 531506
Weekly empty for 542782
Weekly empty for 542823
Weekly empty for 542843
Weekly empty for 521048
Weekly empty for 542830
Weekly empty for 530705
Weekly empty for 539115
Weekly empty for 539288
Weekly empty for 542819
Weekly empty for 542839
Weekly empty for 542850
Weekly empty for 542851
Weekly empty for 542852
Weekly empty for 527005
Weekly empty for 531479
Weekly empty for 531887
Weekly empty for 538563
Weekly empty for 539114
Weekly empty for 542808
Weekly empty for 542812
Weekly empty for 506313
Weekly empty for 523351
Weekly empty for 538926
Weekly empty for 539091
Weekly empty for 532993
Weekly empty for 539116
Weekly empty for 542646
Weekly empty for 530909
Weekly empty for 542857
Weekly empty for 502850
Weekly empty for 531260
Weekly empty for 540140
Weekly empty for 542846
Weekly empty for 542862
Weekly empty for

In [38]:
df_weekly_all.shape

(170808, 22)

In [157]:
# Write BSE Weekly ALL
df_weekly_all.to_csv(os.path.join(os.getcwd(), '..', '5. BTD', 'data', 'df_bse_weekly_all.csv')
                     , encoding='utf-8', index=False)

In [39]:
df_weekly_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170808 entries, 0 to 0
Data columns (total 22 columns):
sc_code       170808 non-null object
yearWeek      170808 non-null object
closeL        170808 non-null float64
closeH        170808 non-null float64
volAvg        170808 non-null object
volHigh       170808 non-null object
daysTraded    170808 non-null object
lastTrdDoW    170808 non-null datetime64[ns]
HSDL          170808 non-null float64
HSDG          170808 non-null float64
HSDLp         170808 non-null float64
HSDGp         170808 non-null float64
first         170808 non-null float64
last          170808 non-null float64
wChng         170808 non-null float64
wChngp        170808 non-null float64
TI            170808 non-null object
volAvgWOhv    170808 non-null object
HVdAV         170808 non-null float64
CPveoHVD      170808 non-null object
lastDVotWk    170808 non-null object
lastDVdAV     170808 non-null float64
dtypes: datetime64[ns](1), float64(12), object(9)
memory usa

### Test Completion Report
**Not satisifed with the timings**

Have to look for alternative approaches as i am going lots of groupby there must be effective way of doing groupby

##### Multiple types of groupby and calculating performance

##### Method 1 - Original

In [21]:
%%time
# close(High/Low), volume average of all days in week, highest volume of the week
df_temp2 = df_temp.groupby(['sc_code', 'yearWeek'])['close'].agg([('L','min'),('H', 'max')]).add_prefix('close').reset_index().copy()

df_temp3 = df_temp.groupby(['sc_code', 'yearWeek'])['volumes'].agg([('Avg','mean'),('High', 'max')]).add_prefix('vol').reset_index().copy()
df_temp3['volAvg']= df_temp3['volAvg'].astype(int)
df_temp3 = pd.merge(df_temp2, df_temp3, how='left', on=['sc_code', 'yearWeek'])
df_weekly = df_temp3.copy()

Wall time: 102 ms


In [22]:
df_weekly.head()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh
0,500002,201913,1313.55,1374.75,11210,27791
1,500002,201914,1379.05,1426.20,16741,34789
2,500002,201915,1408.60,1432.30,9008,21127
3,500002,201916,1433.05,1493.90,23703,50389
4,500002,201917,1473.40,1490.55,13102,27577


##### Method 2 - Removing sorting as "Groupby preserves the order of rows within each group"

In [23]:
%%time
# close(High/Low), volume average of all days in week, highest volume of the week
df_temp2 = df_temp.groupby(['sc_code', 'yearWeek'], sort=False)['close'].agg([('L','min'),('H', 'max')]).add_prefix('close').reset_index().copy()

df_temp3 = df_temp.groupby(['sc_code', 'yearWeek'], sort=False)['volumes'].agg([('Avg','mean'),('High', 'max')]).add_prefix('vol').reset_index().copy()
df_temp3['volAvg']= df_temp3['volAvg'].astype(int)
df_temp3 = pd.merge(df_temp2, df_temp3, how='left', on=['sc_code', 'yearWeek'])
df_weekly = df_temp3.copy()

Wall time: 92 ms


In [24]:
df_weekly.head()

,sc_code,yearWeek,closeL,closeH,volAvg,volHigh
0,500002,201913,1313.55,1374.75,11210,27791
1,500002,201914,1379.05,1426.20,16741,34789
2,500002,201915,1408.60,1432.30,9008,21127
3,500002,201916,1433.05,1493.90,23703,50389
4,500002,201917,1473.40,1490.55,13102,27577


##### Method 3 - all aggregation in single line

In [25]:
%%time
df_temp2 = df_temp.groupby(['sc_code', 'yearWeek']) \
            .agg({"close": ["min",  "max"], "volumes": ["mean", "max"]}).reset_index()
df_temp2.columns = df_temp2.columns.map('_'.join)
df_temp2.rename(columns={"sc_code_":"sc_code", "yearWeek_":"yearWeek", "volumes mean":"volAvg"
                       , "volumes max":"volHigh", "close min":"closeL", "close max":"closeH"}
          , inplace=True)
df_temp2['volAvg']= df_temp3['volAvg'].astype(int)
#df_temp3 = pd.merge(df_temp2, df_temp3, how='left', on=['sc_code', 'yearWeek'])
#df_weekly = df_temp3.copy()

df_temp2.head()

Wall time: 75 ms


,sc_code,yearWeek,volumes_mean,volumes_max,close_min,close_max,volAvg
0,500002,201913,11210.200000,27791,1313.55,1374.75,11210
1,500002,201914,16741.200000,34789,1379.05,1426.20,16741
2,500002,201915,9008.000000,21127,1408.60,1432.30,9008
3,500002,201916,23703.400000,50389,1433.05,1493.90,23703
4,500002,201917,13102.666667,27577,1473.40,1490.55,13102


##### Method 4 - Using assign & groupby and all aggregation in single line

In [26]:
%%time
# Below approach is from user3780389 in https://stackoverflow.com/questions/19078325/naming-returned-columns-in-pandas-aggregate-function

#df_temp2 = df_temp.groupby(['sc_code', 'yearWeek'], as_index=False).agg(closeL=('close','min'),closeM=('close', 'max'))

df_temp2 = df_temp.assign(
    closeL=df_temp['close'],
    closeH=df_temp['close'],
    volAvg=df_temp['volumes'],
    volHigh=df_temp['volumes']
).groupby(['sc_code', 'yearWeek'], as_index=False).agg(dict(closeL=min, closeH=max, volAvg=np.mean, volHigh=max)) \
.reset_index()

df_temp2.drop(['index'], axis=1, inplace=True)

df_temp2['volAvg']= df_temp2['volAvg'].astype(int)
df_weekly = df_temp2.copy()
df_weekly.head()

Wall time: 78.9 ms


,sc_code,yearWeek,closeL,volHigh,volAvg,closeH
0,500002,201913,1313.55,27791,11210,1374.75
1,500002,201914,1379.05,34789,16741,1426.20
2,500002,201915,1408.60,21127,9008,1432.30
3,500002,201916,1433.05,50389,23703,1493.90
4,500002,201917,1473.40,27577,13102,1490.55


##### Method 5 - Trying to remove sc_code (will have to check this as sc_code is not used)

In [27]:
%%time
# close(High/Low), volume average of all days in week, highest volume of the week
df_temp2 = df_temp.groupby(['yearWeek'], sort=False)['close'].agg([('L','min'),('H', 'max')]).add_prefix('close').reset_index().copy()

df_temp3 = df_temp.groupby(['yearWeek'], sort=False)['volumes'].agg([('Avg','mean'),('High', 'max')]).add_prefix('vol').reset_index().copy()
df_temp3['volAvg']= df_temp3['volAvg'].astype(int)
df_temp3 = pd.merge(df_temp2, df_temp3, how='left', on=['yearWeek'])
df_weekly = df_temp3.copy()

Wall time: 77 ms


##### Method 6 - Trying numpy way - Lot of coding might be involved but looks good. Have to try it out

In [28]:
# Below functions are from https://stackoverflow.com/questions/8623047/group-by-max-or-min-in-a-numpy-array

def group_min(groups, data):
    # sort with major key groups, minor key data
    order = np.lexsort((data, groups))
    groups = groups[order] # this is only needed if groups is unsorted
    data = data[order]
    # construct an index which marks borders between groups
    index = np.empty(len(groups), 'bool')
    index[0] = True
    index[1:] = groups[1:] != groups[:-1]
    return data[index]

#max is very similar
def group_max(groups, data):
    # Comment next 3 lines are data is already sorted
    #order = np.lexsort((data, groups))
    #groups = groups[order] #this is only needed if groups is unsorted
    #data = data[order]
    index = np.empty(len(groups), 'bool')
    index[-1] = True
    index[:-1] = groups[1:] != groups[:-1]
    return data[index]

In [29]:
%%time
#df_temp2 = df_temp.groupby(['yearWeek'], sort=False)['close'].agg([('L','min'),('H', 'max')]).add_prefix('close').reset_index().copy()
df_temp2 = df_temp.groupby(['yearWeek'], sort=False)['close'].agg([('H', 'max')]).add_prefix('close').reset_index().copy()
df_temp2.head()

Wall time: 33 ms


,yearWeek,closeH
0,201913,1374.75
1,201914,1426.20
2,201915,1432.30
3,201916,1493.90
4,201917,1490.55


* Just by removing additional column in group by reduced the timing by half

In [30]:
yW = df_temp['yearWeek']
closeVal = df_temp['close']
data_id = np.array(yW)
data_val = np.array(closeVal)

In [31]:
%%time
#data_id = np.array([0,0,0,1,1,1,1,2,2,2,3,3,3,4,5,5,5])
#data_val = np.random.rand(len(data_id))
#print(data_val)
print(group_max(data_id, data_val))

[1374.75 1426.2  1432.3  1477.5  1490.55 1376.95 1392.05 1494.65 1571.
 1564.35 1509.8  1561.1  1603.35 1543.35 1500.8  1414.5  1380.1  1359.65
 1397.7  1359.75 1339.1  1337.85 1320.7  1349.25 1413.3  1545.75 1460.4
 1483.05 1552.65 1496.1  1458.55 1461.65 1467.45 1412.9  1445.7  1444.6
 1486.2  1266.4  1275.75 1284.1  1321.45 1367.25 1303.75 1321.8  1281.45
 1328.05 1234.7  1228.9  1184.55 1193.9  1011.3   958.8   853.1 ]
Wall time: 4 ms


* using Numpy is too fast :D

### Step 5 : Write project write-up

Will be trying to get similar results via Numpy

#### Resources
* [Python - How to check list monotonicity](https://stackoverflow.com/questions/4983258/python-how-to-check-list-monotonicity)

#### Learnings
* Group by only necessary column
* Use .iloc to extract value from specific cell. `last_week = df_weekly2.iloc[-1]['yearWeek'].astype(int)`
* Use .loc when you want to assign a value to a cell
* Emptying a dataframe `df[0:0]`
* Checking if dataframe is empty
```
if df_weekly.empty:
    print('DataFrame is empty!')
```
* FutureWarning in pd.concat, so to avoid that had to add sorting
```
pd.concat([df], sort=True) - Sorts column names
pd.concat([df], sort=False) - Doesn't sort column names
```
* Dataframe to numpy to 2D array
```
df_temp[['yearWeek', 'close']].values
```
* Renaming multiple columns at one
```
df_temp2.rename(columns={"sc_code_":"sc_code", "yearWeek_":"yearWeek", "volumes mean":"volAvg"
                       , "volumes max":"volHigh", "close min":"closeL", "close max":"closeH"}
          , inplace=True)
```

### Simple Tests

**SET A-B**

In [30]:
# This used in weekly_trend_analysis()
list1 = [*range(1, 10, 1)] 
list2 = [*range(5, 15, 1)]
print(list1)
print(list2)

# This returns SET
a = set(list1)
b = set(list2)
print(list(a.difference(b)))

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[1, 2, 3, 4]


**Removing common items**

In [31]:
def remove_common_items(list1, list2): 
    """
    Summary line. 
    Removes the items which are in both the lists

    Parameters: 
    arg1 (list 1)
    arg2 (list 2)

    Returns: 
    list with difference
    
    Sample : print(difference(keep_columns, df_jan.columns))
    """                
    list_dif = [i for i in list1 + list2 if i not in list1 or i not in list2]
    return list_dif

list1 = [*range(1, 10, 1)] 
list2 = [*range(5, 15, 1)]
print(list1)
print(list2)
print(remove_common_items(list1, list2))

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[1, 2, 3, 4, 10, 11, 12, 13, 14]
